# Google gmaps

## 6.5.2 Create Heatmaps for Weather Parameters

In [1]:
# Import dependencies
import pandas as pd
import gmaps
import requests

# import api key
from config import g_key

# import widgets
import ipywidgets as widgets

In [2]:
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Cabedelo,BR,2021-09-24 21:30:50,-6.9811,-34.8339,79.47,81,11,13.82
1,1,Buraydah,SA,2021-09-24 21:30:50,26.3260,43.9750,90.05,19,9,10.65
2,2,Qaqortoq,GL,2021-09-24 21:30:50,60.7167,-46.0333,39.70,51,1,7.02
3,3,Isangel,VU,2021-09-24 21:30:51,-19.5500,169.2667,73.51,83,90,0.00
4,4,Kavaratti,IN,2021-09-24 21:30:51,10.5669,72.6420,81.50,79,96,14.12


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

### Create a Maximum Temperature Heatmap

In [4]:
# Configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [5]:
def create_heat_map(locations, weights, marker_info=None):
    """
    create new heat map with gmaps for locations and weights passed in
    
    args:
        locations: DataFrame with two columns [["Lat", "Lng"]]
        weights: List of max temperatures or humidity, or another weather metric
    """ 
    # create figure
    fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
    
    heat_layer = gmaps.heatmap_layer(locations, weights, dissipating=False, max_intensity=300, point_radius=4)
        
    # add the heatmap layers
    fig.add_layer(heat_layer)

    # add marker layer if marker_info argument is provided
    if (marker_info != None):
        marker_layer = gmaps.marker_layer(locations, info_box_content=marker_info)
        fig.add_layer(marker_layer)

    
    # call figure to plot the data
    return fig

In [6]:
# Get the maximum temperature
max_temp = city_data_df["Max Temp"]
temps = [max(temp,0) for temp in max_temp]

In [7]:
# Heatmap of temperature
# Get latitude and longitude
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature
max_temp = city_data_df["Max Temp"]

# Assign the figure variable
fig = gmaps.figure()

# Assign the heatmap variable
# Adjust Heatmap Zoom, Intensity, and Point Radius
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)

# Add the heatmap layer
fig.add_layer(heat_layer)

# call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

### Create Percent Humidity Heatmap

In [8]:
# Heatmap of percent hunidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create Wind Speed Heatmap

In [9]:
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]

fig = create_heat_map(locations, wind)
fig

Figure(layout=FigureLayout(height='420px'))

## 6.5.3 Get Vacation Criteria

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Cabedelo,BR,2021-09-24 21:30:50,-6.9811,-34.8339,79.47,81,11,13.82
4,4,Kavaratti,IN,2021-09-24 21:30:51,10.5669,72.6420,81.50,79,96,14.12
8,8,Puri,IN,2021-09-24 21:30:52,19.8000,85.8500,79.32,83,98,2.42
10,10,Puerto Ayora,EC,2021-09-24 21:28:10,-0.7393,-90.3518,76.96,78,64,9.89
12,12,Cayenne,GF,2021-09-24 21:30:53,4.9333,-52.3333,84.24,82,20,1.99
15,15,Lompoc,US,2021-09-24 21:30:53,34.6391,-120.4579,84.13,79,1,11.50
18,18,Georgetown,MY,2021-09-24 21:30:13,5.4112,100.3354,81.90,91,20,3.00
19,19,Atuona,PF,2021-09-24 21:30:54,-9.8000,-139.0333,77.85,78,74,16.44
22,22,Butaritari,KI,2021-09-24 21:30:55,3.0707,172.7902,83.80,74,12,13.56
25,25,Avarua,CK,2021-09-24 21:30:56,-21.2078,-159.7750,77.05,53,92,9.22


In [13]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       1
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [14]:
preferred_cities_df.count()

City_ID       162
City          162
Country       161
Date          162
Lat           162
Lng           162
Max Temp      162
Humidity      162
Cloudiness    162
Wind Speed    162
dtype: int64

In [15]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Cabedelo,BR,79.47,-6.9811,-34.8339,
4,Kavaratti,IN,81.50,10.5669,72.6420,
8,Puri,IN,79.32,19.8000,85.8500,
10,Puerto Ayora,EC,76.96,-0.7393,-90.3518,
12,Cayenne,GF,84.24,4.9333,-52.3333,
15,Lompoc,US,84.13,34.6391,-120.4579,
18,Georgetown,MY,81.90,5.4112,100.3354,
19,Atuona,PF,77.85,-9.8000,-139.0333,
22,Butaritari,KI,83.80,3.0707,172.7902,
25,Avarua,CK,77.05,-21.2078,-159.7750,


Place Search API doc:
https://developers.google.com/maps/documentation/places/web-service/search#PlaceSearchRequests%0D%0A

In [16]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

In [17]:
# https://requests.kennethreitz.org/en/master/api/#main-interface

In [18]:
params = {
    "key": g_key,
    "type": "lodging",
    "radius": 5000
}

In [19]:
# Iterate through DataFrame
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key 
    params["location"] = f"{lat},{lng}"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"] 
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162 entries, 0 to 542
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        162 non-null    object 
 1   Country     161 non-null    object 
 2   Max Temp    162 non-null    float64
 3   Lat         162 non-null    float64
 4   Lng         162 non-null    float64
 5   Hotel Name  162 non-null    object 
dtypes: float64(3), object(3)
memory usage: 12.9+ KB


In [21]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Cabedelo,BR,79.47,-6.9811,-34.8339,Hotel Pigalle
4,Kavaratti,IN,81.50,10.5669,72.6420,Hotels in Lakshadweep Islands
8,Puri,IN,79.32,19.8000,85.8500,OYO 1577 Hotel Shree Hari Grand
10,Puerto Ayora,EC,76.96,-0.7393,-90.3518,Finch Bay Galapagos Hotel
12,Cayenne,GF,84.24,4.9333,-52.3333,Hôtel Le Dronmi
15,Lompoc,US,84.13,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
18,Georgetown,MY,81.90,5.4112,100.3354,Cititel Penang
19,Atuona,PF,77.85,-9.8000,-139.0333,Villa Enata
22,Butaritari,KI,83.80,3.0707,172.7902,Isles Sunset Lodge
25,Avarua,CK,77.05,-21.2078,-159.7750,Paradise Inn


### Add a pop-up marker for each city

In [22]:
"""
Description List: <dl>
Description Term: <dt>
Description Definition: <dd>
Description Term: <dt>
Description Definition: <dd>
"""

'\nDescription List: <dl>\nDescription Term: <dt>\nDescription Definition: <dd>\nDescription Term: <dt>\nDescription Definition: <dd>\n'

In [23]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [24]:
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [25]:
hotel_info[0]

'\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Pigalle</dd>\n<dt>City</dt><dd>Cabedelo</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Max Temp</dt><dd>79.47 °F</dd>\n</dl>\n'

In [26]:
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

fig = create_heat_map(locations, max_temp, hotel_info)
fig

Figure(layout=FigureLayout(height='420px'))